In [4]:
import os
import logging 

In [19]:
#设置日志记录 LLM --prompt question + response 记录
def setup_logger():
    """Setup logger to always print time and level."""
    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S')
    logging.getLogger().setLevel(logging.INFO)  # logging.DEBUG

#indent区分
def log_w_indent(text, indent, flag ,symbol='>>'):
    """Log and add indent."""
    ind2col = {i: f"\x1b[{a}" for i, a in enumerate([
        '1m', '31m', '33m', '34m', '35m'])}
    reset = "\x1b[0m"

    if indent > 0:
        if flag == 'Question':
            logging.info('  Q:'+
                ind2col[indent] + (indent * 2) * symbol + ' ' + text + reset)
        elif flag =='Response':
            logging.info('  LLM-A:'+
                ind2col[indent] + (indent * 2) * symbol + ' ' + text + reset)

    else:
        logging.info(ind2col[indent] + text + reset)

setup_logger()
log_w_indent('This is a test message',2,'Question')
log_w_indent('Another message without indent', 0,'Response')

INFO:root:  Q:>>>>>>>> This is a test message
INFO:root:Another message without indent


In [37]:
#配置openai客户端
from openai import OpenAI
#from dotenv import load_dotenv
import os
import json
#config.json 进行配置用户openai账户 
with open(r'neuron_seg_human\llm_neuron_seg\utils\llm\llm_config.json') as config_file:
    config = json.load(config_file)

CLIENT = OpenAI(base_url=config['OPENAI_API_BASE'],
                api_key=config['OPENAI_API_KEY'])

In [ ]:
import hashlib
from collections import defaultdict

class SpoofData:
    def __getitem__(self, item):
        return f'<{item}>'
def md5hash(string):
    return int(hashlib.md5(string.encode('utf-8')).hexdigest(), 16)

class LLMResponse:
    def __init__(self,ini_task:str,n_regenerate,reset,reset_stages):
        self.ini_task_question=ini_task
        self.n_regenerate=n_regenerate
        self.reset=reset
        self.reset_stages=reset_stages
        self.export_predict_response = defaultdict(defaultdict(list).copy)
        
    #indent用于日志记录 stepid:步骤id  reuse:是否进行再次输入迭代
    def llw_w_log(self, prompt, indent,stepidx, stage,reuse=False):
        #reset.stages 重置阶段列表     拿到需要重新run的stepidx
        if (stage in self.reset_stages) and (reset := self.reset.get(stepidx, {})):
            if self.accept_restore_failure and not len(reset[md5hash(prompt)]):
                logging.warning('Spoofing aborted!')
                predict_response =llm_w_log(prompt, indent)
            else:
                # Re-Use predictions from previous runs.
                #def log_w_indent(text, indent, flag ,symbol='>>'):
                log_w_indent(f'Spoofed Input: {prompt}', indent,flag='Question')
                predict_response= reset[md5hash(prompt)].pop(0)
                log_w_indent(f'Spoofed Output: {predict_response}', indent, flag='Question',symbol='xx')
        # elif reuse and (previous := self.export_predictions.get(stepidx, {}).get(md5hash(prompt), False)):
        #     # Usually don't want to do this, but for some yes-no questions,
        #     # there is basically no variance and it saves significant compute
        #     # to just repeat the previous answer (e.g. for entailment checks).
        #     log_w_indent(f'Reused Input: {prompt}', indent)
        #     prediction = previous[0]
        #     log_w_indent(f'Reused Output: {prediction}', indent, symbol='xx')
        else:
            predict_response = llm_w_log(prompt, indent)

        self.export_predict_response[stepidx][md5hash(prompt)].append(predict_response)

        return predict_response

    def gen_ini_task_decription(self, data):
        del data
        #return 'Please list the specific factual propositions included in the answer above. Be complete and do not leave any factual claims out. Provide each claim as a separate sentence in a separate bullet point.'
        return 'For this neuroscience image task, please develop a programming task solution plan and divide your thinking into three parts: "# # Task Objectives and Function Description", "# # # Algorithm Steps and Implementation Details", and "# # Specific Constraints". There is no need to write scripts and Provide each claim as a separate sentence in a separate bullet point.'
    
    def gen_task_step(self,data):
        step_description =data
        self.step_description = step_description
        return step_description
    
    def process_step_description(self):
        step_prompt_list=self.step_description.split()

        return step_prompt_list

    #生成step列表 eachstep 进行prompt修饰后 进行llm输入，获取处理代码
    def manual_modify_step(self,data) -> list:
        del data

       
        step_prompt_list = process_step_description(data)

        ###human modify step_list###


        return step_prompt_list
    
    
    def get_all_prompts_for_log(self):
        # Spoof data to log prompting format.
        data = SpoofData()
        prompts = dict(
            gen_ini_task_decription=self.gen_ini_task_decription(data),
            task_step_lsit= manual_modify_step(data))
            # base_gen_questions=self.base_gen_questions(data),
            # base_answer_question=self.base_answer_question(data),
            # base_equivalence=self.base_equivalence(data))

        return prompts

    def gen_questions(self, data):
        del data
        raise

    def answer_question(self, data):
        del data
        raise

 


In [45]:
def get_sentences(response):
    """Extract scripts & descriptions from LLM-response."""
    # Some manual exceptions for sentence extraction.

    facts = response.replace('Ph.D.', 'PhD').replace('\n', ' ').split('. ')
    # facts = [f.strip() + '.' if i < len(facts) - 1 else f.strip() for i, f in enumerate(facts)]
    for i in facts:
        print(i)

def get_sentences(response):
    """Extract scripts & descriptions from LLM-response."""
    # Some manual exceptions for sentence extraction.
    
    # facts = response.replace('Ph.D.', 'PhD').replace('\n', ' ').split('. ')
    # facts = [f.strip() + '.' if i < len(facts) - 1 else f.strip() for i, f in enumerate(facts)]
    for i in facts:
        print(i)

from tenacity import (retry, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=10))
def chat_llm(prompt):
    """Predict with GPT-4 model."""

    if isinstance(prompt, str):
        messages = [
            {"role": "user", "content": prompt},
        ]
    else:
        messages = prompt

    output = CLIENT.chat.completions.create(
        model='gpt-4o',
        messages=messages,

    )
    response = output.choices[0].message.content
    return response

def llm_w_log(prompt):
    """ log inputs and outputs."""
    log_w_indent(f'Input: {prompt}', indent=1 , flag='Question')
    response = chat_llm(prompt)
    ##处理response
    log_w_indent(f'Output: {response}', indent=2, symbol='xx',flag='Response')
    return response
llm_w_log('how to trace and restruct human neuron structure based on 3d tif images?')

INFO:root:  Q:>>>> Input: how to trace and restruct human neuron structure based on 3d tif images?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:  LLM-A:xxxxxxxx Output: Tracing and reconstructing human neuron structures from 3D TIFF (Tagged Image File Format) images is a complex process that typically involves image processing, segmentation, and 3D visualization techniques. Here are the general steps you can follow, broken down into core components and tools you will need:

### Materials and Tools
- **3D TIFF images**: High-resolution images of the neuron structures.
- **Image analysis software**: For example, `ImageJ/Fiji`, `Neuroph`, `ITK-SNAP`, or `Imaris`.
- **Neuron tracing software**: `NeuronStudio`, `NeuroLucida`, `Simple Neurite Tracer` (a plugin for ImageJ), or `Horton`.
- **Programming Environment**: For advanced customization, tools like Python with libraries such as `scikit-image`, `numpy`, and `matplotlib`, or MATLAB.

'Tracing and reconstructing human neuron structures from 3D TIFF (Tagged Image File Format) images is a complex process that typically involves image processing, segmentation, and 3D visualization techniques. Here are the general steps you can follow, broken down into core components and tools you will need:\n\n### Materials and Tools\n- **3D TIFF images**: High-resolution images of the neuron structures.\n- **Image analysis software**: For example, `ImageJ/Fiji`, `Neuroph`, `ITK-SNAP`, or `Imaris`.\n- **Neuron tracing software**: `NeuronStudio`, `NeuroLucida`, `Simple Neurite Tracer` (a plugin for ImageJ), or `Horton`.\n- **Programming Environment**: For advanced customization, tools like Python with libraries such as `scikit-image`, `numpy`, and `matplotlib`, or MATLAB.\n- **High-performance Computing (HPC)** (if dealing with very large datasets).\n\n### Steps to Trace and Reconstruct Neuron Structures:\n\n#### 1. Pre-processing the Image:\nBefore segmentation and tracing, enhance th